# Numerical types and error
Click [here](https://datahub.berkeley.edu/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fberkeley-physics%2FPython-Tutorials&urlpath=tree%2FPython-Tutorials%2F2+-+Intermediate%2F1+-+Numerical+types.ipynb&branch=master) to open this notebook in the DataHub.

## Learning objectives
By the end of this tutorial, you will be able to:
- Distinguish and convert between numerical data types
- Understand the concepts of overflow, underflow and roundoff error
- Compare floating-point numbers and check for equality
- Recognise and avoid sources of numerical error
- Recognise and avoid errors caused by type mismatch

## Relevant documentation
- [Python data types](https://docs.python.org/3/library/stdtypes.html)
- [NumPy data types](https://numpy.org/doc/stable/user/basics.types.html)
- [Python float limitations tutorial](https://docs.python.org/3/tutorial/floatingpoint.html)

## Numerical types
You have seen native Python types, e.g. when inputting numbers directly into Python (these are known as literals). Determine the type of each of the variables defined below. (You might find the function `type(...)` useful.)

In [ ]:
x = 3
y = 9.5
z = -4+3j

Very often, we'll be using `NumPy`, which is mostly a Python wrapper for C++ code. This means that while the code runs much faster, the numbers are not Python types, but C++ types, which behave differently. Find the types of the variables `a`, `b`, `c`, and `d`:

In [ ]:
import numpy as np

zeros_array = np.zeros(5)
linear_array = np.linspace(-1,1,10)
example_list = [1,2,3,4]
example_array = np.array(example_list)

a = zeros_array[0]
b = linear_array[0]
c = example_list[0]
d = example_array[0]

Are these types different from those of native Python? Why?

## Integers
Integers are typically stored with no loss. If we choose to store an integer using $n$ bits, we can map these $n$ bits to $2^n$ integer values, typically from $-2^{n-1}$ to $2^{n-1}-1$, inclusive. For example, in a byte (8 bits), we can store one of 256 integers, which are conventionally chosen to be the range from -128 to 127.

Numpy allows you to choose between 8-bit, 16-bit, 32-bit, 64-bit, and 128-bit integers, resorting to 64-bit integers by default on most systems. You can choose the data type when constructing an array, or explicitly convert numbers and arrays to another data type if required.

Run the following cells and explain what is happening.

In [ ]:
a = np.array([-120,30,200,128], dtype=np.int8)
a

In [ ]:
a+100, a-100

With 64-bit integers (the usual default), the minimum and maximum integers are $-2^{63}=-9223372036854775808$ and $2^{63}-1=9223372036854775807$. (It is useful to remember than $2^{10}=1024\approx1000= 10^3$, so $2^{63}\approx 10^{19}$.)

In [ ]:
b = np.array([2])
b.dtype #to check type of array (rather than type of element)

In [ ]:
b**62, b**63

You might have noticed that C++-style integer arithmetic is equivalent to modular arithmetic, i.e. values differing by $2^n$ are indistinguishable. 

Python, on the other hand, automatically allots more bits to large integers, ensuring that integers remain exact and not modular. In line with the Python philosophy, this provides convenience at the cost of transparency and performance.

In [ ]:
2**64, b**64

So, when using NumPy integers, we must be careful not to have integers that are too large, since these may wrap around __without throwing an error__, which means you may not notice when your program goes wrong.

## Floating-point numbers
It may be intuitive to you that a digital computer with a finite amount of space cannot store an element of a continuous set. In 8 bits, we can store only 256 distinct values, which is far smaller than the "number" (the "size" of the set of) of real numbers. One data type that is meant to encode real numbers is called the _floating-point number,_ or _float_. The strategy here is to allot one bit for sign, $m$ bits for a binary fraction, and the remaining $n-m-1$ bits for an exponent. This means that the $2^{n}$ possible values map to, $$(-1)^s \left(1+\frac{f}{2^m}\right)2^e,$$ where $s$ represents the sign bit, $f$ represents the binary fraction (an integer between $0$ and $2^m-1$), and $e$ represents the exponent (typically between $-2^{n-m-2}$ and $2^{n-m-2}-1$). (In practice, several "special" values like `inf` and `nan` are reserved.)

For the 64-bit float, which is the typical default for both Python and NumPy, $m=52$. This means that 11 bits are reserved for the exponent, i.e. it has values from -1024 to 1023. Thus the smallest float (in magnitude) is $2^{-1074}\approx 10^{-325}$ (since for small numbers the leading 1 of the binary fraction can be ignored), and the largest float is slightly below $2^{1024}\approx 10^{308}$. When we attempt to do floating-point arithmetic whose result is outside of the range of numbers representable by the floats we are using, that is known as _overflow_ or _underflow_ (for numbers that are too large and small respectively). Run the following cells and explain the results:

In [ ]:
x = 2.0
y = np.array(x, dtype=np.float64)
z = y.astype(np.float128) #copying y to z, but changing type

In [ ]:
x**-1074, x**-1075 #try with y and z

In [ ]:
print(x**1023*1.9999)
print(x**1024) 
#try with y and z

Since we are using a finite list of values to represent an infinite set of numbers, each float value corresponds to a range of numbers. Specifically, it specifies values within a relative range of $2^{-m}$. For example, the closest float representing the numbers $1\pm 2^{-53}$ is 1. This is known as _roundoff error_. It implies that all floats are accurate only to a factor of $2^{-52}\approx 10^{-16}$.

In [ ]:
x = 2**-53
y = 1.0

In [ ]:
x == 0

In [ ]:
x + y == y

This means that when we add numbers with different magnitudes, we lose accuracy. This also holds for small differences between large numbers (for example, if we want to estimate the derivative of a function by comparing its values at neighbouring points, we might lose accuracy by taking points that are too close together).

In [ ]:
x = 1.23456789
y = 9.87654321
print(y-x)

z = 10**10
print((z+y)-(z+x))

This also means that testing floats for equality can be tricky; consider the following example.

In [ ]:
0.7 * 0.4 == 0.28

The usual strategy to do this is to check if the two floats agree to some (arbitrary) precision. Why is it better to use a relative tolerance than an absolute tolerance? (E.g. compare the difference between 1000000000000000.0 and 1000000000000000.1 to that between 1.0 and 1.001) Implement the following function.

In [ ]:
def float_equal(x, y, tol=1e-6):
    """Returns True if x/y is within the range [1-tol,1+tol], False otherwise."""
    #implement me
    return False
    
float_equal(0.7*0.4, 0.28)

You might notice that all floats are exact binary fractions, and wonder why, since all integers (not just binary ones) can be represented exactly, all rational numbers are not represented exactly. You can further pursue this idea in the tutorial on abstract objects.

Now try calculating $n!$ for various $n$. If we want exact answers, should we use floats or ints? Try both and judge the error (if any). If you use a NumPy int, what is the highest $n$ whose factorial you can calculate exactly? How about a Python int? What is the highest $n$ whose factorial you can compute as a float, before running into overflow? Calculate $\log n!$ for even higher $n$. How would you use this in further calculations, e.g. in finding the number of combinations of $m$ objects from $n$ choices? Which data type computes faster?

In general, going to log-space is a good idea if you are working with extremely large numbers (e.g. calculate the entropy instead of the number of microstates).

## Changing types
Python (and NumPy) automatically (implicitly) converts types where necessary to preserve information; see the examples below.

In [ ]:
print(2+0.5)
print(4*2)
print(4/2)
print(np.array([4,4])/np.array([1,2]))

How would you use this feature, e.g. to convert the integer 2 to a float?

You can also explicitly change types using the `int(...)` and `float(...)` functions, or the NumPy `asType` method as seen above.

In [ ]:
int(3.5), int(round(3.5))

In [ ]:
float(4)

The Boolean type (True/False values) converts to 1/0.

In [ ]:
type(False)

In [ ]:
int(False), float(True)

In [ ]:
bool(12.3), bool(0)

Implement the following function using as little code as possible (_code golf_):

In [ ]:
def bool_convert(b):
    """Returns the integer 27 if b is True, and 0 if b is False"""

bool_convert(True), bool_convert(False)

Python is not typed, which means variables can be reassigned to any type, and parameter types don't have to be specifield. In the `bool_convert(b)` function above, we have assumed that `b` is a Boolean, but this is not enforced: we may pass a float or an int, and the function will still attempt to run.

In [ ]:
bool_convert(10), bool_convert(0.67)

While this weak type enforcement provides greater convenience by allowing us to write functions that accept several input types, it can lead to errors when we use an unexpected type without realising, since some input types will cause the program to crash or have unexpected behaviour. 

In [ ]:
bool_convert("hello")

To gracefully handle unexpected input types, you can explicitly check the type of each relevant parameter. The `assert` statement (find the relevant docs!) throws an error when a condition isn't met. Try enforcing the _precondition_ that the argument to the `bool_convert` function must be a Boolean.

In [ ]:
x = 3
assert type(x) == float, "x is not a float!"